In [ ]:
pip install lightgbm holidays

In [44]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import holidays
import time

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor


s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/train.csv'
file_key_2 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/test.csv'
file_key_3 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/sample_submission.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

## Reading data-files
train = pd.read_csv(file_content_stream_1)
train['date'] = pd.to_datetime(train['date'], format = '%Y-%m-%d')

test = pd.read_csv(file_content_stream_2)
test['date'] = pd.to_datetime(test['date'], format = '%Y-%m-%d')

submission = pd.read_csv(file_content_stream_3)

## Basic feature engineering 
train['weekday'] = train['date'].dt.dayofweek
train['month'] = train['date'].dt.month
train['weekend'] = np.where(train['weekday'] >= 5, 1, 0)
train['dayOfMonth'] = train['date'].dt.day
train['dayOfYear'] = train['date'].dt.dayofyear

test['weekday'] = test['date'].dt.dayofweek
test['month'] = test['date'].dt.month
test['weekend'] = np.where(test['weekday'] >= 5, 1, 0)
test['dayOfMonth'] = test['date'].dt.day
test['dayOfYear'] = test['date'].dt.dayofyear

In [45]:
test.head()

,row_id,date,country,store,product,weekday,month,weekend,dayOfMonth,dayOfYear
0,70128,2021-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques,4,1,0,1,1
1,70129,2021-01-01,Belgium,KaggleMart,Kaggle Getting Started,4,1,0,1,1
2,70130,2021-01-01,Belgium,KaggleMart,Kaggle Recipe Book,4,1,0,1,1
3,70131,2021-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,4,1,0,1,1
4,70132,2021-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques,4,1,0,1,1


In [46]:
from sklearn.preprocessing import LabelEncoder

all_data = pd.concat([train, test], axis = 0)

le = LabelEncoder()
cols = ['country', 'store', 'product']
for col in cols:
    le = LabelEncoder()
    all_data[col] = le.fit_transform(all_data[col])

In [47]:
all_data = all_data.drop(['date', 'row_id'], axis = 1)
train = all_data.iloc[:70128,:]
test = all_data.iloc[70128:,:].drop(['num_sold'], axis = 1)

X = train.drop(['num_sold'], axis = 1)
Y = np.log(train['num_sold'])

In [56]:
# data_temp = train[(train['country'] == 'Belgium') & (train['store'] == 'KaggleMart') & (train['product'] == 'Kaggle Advanced Techniques')].reset_index(drop = True)

# X = data_temp.drop(columns = ['row_id', 'date', 'country', 'store', 'product', 'num_sold'], axis = 1)
# Y = np.log(data_temp['num_sold'])


t1 = time.time()
kf = KFold(n_splits = 4, shuffle = True, random_state = 34)
off = np.zeros(len(train))
score_list_lgb = []
test_preds_lgb = []
fold = 1

for train_index, test_index in kf.split(X, Y):
    
    ## Splitting the data
    X_train , X_val = X.iloc[train_index], X.iloc[test_index]  
    Y_train, Y_val = Y.iloc[train_index], Y.iloc[test_index]    
    
    print("X_train shape is :", X_train.shape, "X_val shape is", X_val.shape)
    y_pred_list = []
    
    model_lgb = LGBMRegressor(n_estimators = 3000, 
                              learning_rate = 0.01,
                              num_leaves = 37,
                              max_depth = 9, 
                              lambda_l1 = 3, 
                              lambda_l2 = 1, 
                              bagging_fraction = 0.95, 
                              feature_fraction = 0.96)

    model = model_lgb.fit(X_train, Y_train)
    result = model_lgb.predict(X_val)
    
    result = pd.DataFrame(result)
    result.iloc[:, 0] = [0 if i <= 0 else i for i in result.iloc[:,0]]
    
    score = np.sqrt(mean_squared_error(Y_val, result))
    print('Fold ', str(fold), ' result is:', score, '\n')
    score_list_lgb.append(score)

    test_preds_lgb.append(model_lgb.predict(test))
    fold +=1

t2 = time.time()
print("LGBM model with cross validation take : {:.3f} sn.".format(t2-t1))

X_train shape is : (52596, 8) X_val shape is (17532, 8)
[LightGBM] [Warning] lambda_l1 is set=3, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3
[LightGBM] [Warning] feature_fraction is set=0.96, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.96
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[LightGBM] [Warning] bagging_fraction is set=0.95, subsample=1.0 will be ignored. Current value: bagging_fraction=0.95
Fold  1  result is: 0.1916374533614213 

X_train shape is : (52596, 8) X_val shape is (17532, 8)
[LightGBM] [Warning] lambda_l1 is set=3, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3
[LightGBM] [Warning] feature_fraction is set=0.96, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.96
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[LightGBM] [Warning] bagging_fraction is set=0.95, subsample=1.0 will be ignor

In [57]:
mean = sum(score_list_lgb) / len(score_list_lgb)
variance = sum([((x - mean) ** 2) for x in score_list_lgb]) / len(score_list_lgb)
res = variance ** 0.5
print("Cross validation mean score:", sum(score_list_lgb) / len(score_list_lgb))
print("Cross validation score's Standart deviation is:", res)

Cross validation mean score: 0.18857231445736194
Cross validation score's Standart deviation is: 0.0020893474714655143


In [58]:
test_preds_lgb = pd.DataFrame(test_preds_lgb)
print(test_preds_lgb.shape)

test_preds_lgb = test_preds_lgb.mean(axis = 0)
print(test_preds_lgb.head(5))

(4, 17520)
0    6.295334
1    6.011356
2    5.847804
3    6.332231
4    5.242429
dtype: float64


In [59]:
submission['num_sold'] = np.exp(test_preds_lgb)
submission.to_csv('LightGBM_submission_4.csv', index = False)
submission.head()

,row_id,num_sold
0,70128,542.036830
1,70129,408.036342
2,70130,346.472618
3,70131,562.409724
4,70132,189.128937
